# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [97]:
# import libraries
import sqlalchemy as sql
import pandas as pd
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dirk\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dirk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dirk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dirk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
dbname = 'disaster_response_db'
tbname = 'dr_messages_tbl'

# load data from database
conn = sql.create_engine('sqlite:///{}.db'.format(dbname)).connect()
df = pd.read_sql_table(tbname, con=conn)

In [66]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,direct,news
count,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,...,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000,26177.000000
mean,15226.345265,0.773580,0.170531,0.004470,0.414104,0.079459,0.050044,0.027658,0.017993,0.032815,...,0.278336,0.082095,0.093212,0.010773,0.093670,0.020170,0.052565,0.193414,0.410551,0.497994
std,8827.039557,0.435345,0.376106,0.066707,0.492576,0.270459,0.218040,0.163994,0.132928,0.178156,...,0.448188,0.274515,0.290734,0.103234,0.291375,0.140586,0.223168,0.394982,0.491943,0.500006
min,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7449.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15664.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22925.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,30265.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [67]:
#Child alone column only has zeros
print ('Sum of child_alone column: ',df['child_alone'].sum())
df.drop(['child_alone'], axis=1, inplace=True)

Sum of child_alone column:  0


In [68]:
#only the related column has numbers other than 1 and 0, this may be an error, reduce the 2 in this coulmn to a 1
df.groupby('related').count()

,id,message,original,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,direct,news
related,,,,,,,,,,,,,,,,,,,,,
0,6115,6115,3390,6115,6115,6115,6115,6115,6115,6115,...,6115,6115,6115,6115,6115,6115,6115,6115,6115,6115
1,19874,19874,6631,19874,19874,19874,19874,19874,19874,19874,...,19874,19874,19874,19874,19874,19874,19874,19874,19874,19874
2,188,188,132,188,188,188,188,188,188,188,...,188,188,188,188,188,188,188,188,188,188


In [69]:
df['related'] = df['related'].apply(lambda x: 1 if x == 2 else x)

In [70]:
X = df.message
y = df.drop(['id', 'message', 'original'], axis=1)

### 2. Write a tokenization function to process your text data

In [71]:
def tokenize(text):
    """
    Input:
    text (str): text from all messages
       
    Returns:
    clean_tokens (list): list of words into numbers of same meaning
    """    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    detected_urls = re.findall(url_regex, text)
    
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")  
        
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    text = re.sub(r'[0-9]', " ", text)
    
    tokens = nltk.word_tokenize(text)

    #tokens = [word for word in tokens if not word in stopwords.words('english')]

    clean_tokens = []
    
    lemmatizer = nltk.WordNetLemmatizer()
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [72]:
#Test tokenize function
print (tokenize(df.message[5]))

['information', 'about', 'the', 'national', 'palace']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [84]:
def basic_pipeline():
    '''
    Creates a pipeline object with a countvectorizer, tfidtransformation and a
    multioutputclassifier with a random forest classifier estimator

            Input:
                    None

            Returns:
                    pipeline object: An initialized pipeline object
    '''    
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])
    
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = basic_pipeline()
model.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000149179B9550>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [86]:
y_predicted_test = model.predict(X_test)

In [87]:
print (classification_report(y_test, y_predicted_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.97      0.89      5006
               request       0.90      0.45      0.60      1159
                 offer       0.00      0.00      0.00        33
           aid_related       0.78      0.63      0.70      2716
          medical_help       0.74      0.06      0.10       509
      medical_products       0.80      0.12      0.21       328
     search_and_rescue       0.89      0.04      0.08       186
              security       0.00      0.00      0.00       130
              military       0.83      0.02      0.04       234
                 water       0.88      0.29      0.43       424
                  food       0.86      0.46      0.60       726
               shelter       0.90      0.27      0.41       594
              clothing       0.58      0.07      0.13        95
                 money       1.00      0.02      0.04       139
        missing_people       0.00      

C:\Users\Dirk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dirk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dirk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
#Test the model with a simple message to see if useful catergories are returned
msg = ['Please help us, there is a massive storm approaching and we have no shelter. There are fires all around us. Its flooding too']
test = model.predict(msg)
print (y_train.columns.values[(test.flatten()==1)])

['related' 'request' 'aid_related' 'weather_related' 'floods'
 'direct_report' 'direct']


### 6. Improve your model
Use starting a verb extractor and use an adaboost classifier for the final model estimator. 

In [90]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))

            # index pos_tags to get the first word and part of speech tag
            
            if not len(pos_tags): 
                return False
            first_word, first_tag = pos_tags[0]
            
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True

        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [91]:
def final_pipeline():
    '''
    Creates a pipeline object with a countvectorizer, tfidtransformation, starting verb extractor and a
    multioutputclassifier with a ada boost classifier estimator

            Input:
                    None

            Returns:
                    pipeline object: An initialized pipeline object
    '''
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            
            ('starting_verb', StartingVerbExtractor())
        ])),
        
        ('classifier', MultiOutputClassifier(AdaBoostClassifier()))
    ])
    
    return pipeline

In [92]:
model_improved = final_pipeline()
model_improved.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x00000149179B9550>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('classifier',
                 MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [93]:
y_predicted_test_improved = model_improved.predict(X_test)

In [94]:
print (classification_report(y_test, y_predicted_test_improved, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.82      0.95      0.88      5006
               request       0.78      0.54      0.64      1159
                 offer       0.08      0.03      0.04        33
           aid_related       0.76      0.58      0.66      2716
          medical_help       0.57      0.30      0.39       509
      medical_products       0.60      0.33      0.43       328
     search_and_rescue       0.57      0.21      0.31       186
              security       0.27      0.05      0.08       130
              military       0.57      0.30      0.40       234
                 water       0.73      0.62      0.67       424
                  food       0.81      0.69      0.74       726
               shelter       0.78      0.52      0.63       594
              clothing       0.70      0.41      0.52        95
                 money       0.56      0.26      0.35       139
        missing_people       0.50      

C:\Users\Dirk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Dirk\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [95]:
#Test the model with a simple message to see if useful catergories are returned
msg = ['Please help us, there is a massive storm approaching and we have no shelter. There are fires all around us. Its flooding too']
test = model_improved.predict(msg)
print (y_train.columns.values[(test.flatten()==1)])

['related' 'request' 'aid_related' 'water' 'shelter' 'weather_related'
 'floods' 'storm' 'fire' 'direct_report' 'direct']


The model returns more accurate categories, as previously the model weas not able to categorize the message to include fire

Use grid search to determine optimim values of parameters for further improvement

In [18]:
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000),
    'features__text_pipeline__tfidf__use_idf': (True, False)
    }

cv = GridSearchCV(model_improved, param_grid=parameters, scoring='f1_micro')
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000016922792EE0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__min_samples_split': [2, 5, 10],
                         'clf__estimator__n_estimators': [10, 25]},
             scoring='f1_micro')

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [98]:
filename = 'classification_model.sav'
pickle.dump(model_improved, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.